In [6]:
import pandas as pd

# output-ave.txtからデータを読み込む
df_all = pd.read_csv('output-ave.txt', sep='\t')



In [7]:

# データフレームの例
# df_all = pd.DataFrame(...)

# Method と Place の具体的な意味をラベルに追加する
method_labels = {
    -1: '音無し',
    0: '離散指数',
    1: '連続指数',
    2: '連続線形',
    3: '離散線形',


}

place_labels = {
    0: '(0.25,-0.1,0)',
    1: '(0.25,0.1,0)',
    2: '(0.5,-0.1,0)',
    3: '(0.5,0.1,0)',
    4: '(0.25,-0.1,-0.1)',
    5: '(0.25,0.1,-0.1)',
    6: '(0.5,-0.1,-0.1)',
    7: '(0.5,0.1,-0.1)'
}

# Method と Place のラベルをデータフレームに追加
df_all['Method_Label'] = df_all['Method'].map(method_labels)
df_all['Place_Label'] = df_all['Place'].map(place_labels)

In [8]:
from itertools import combinations
import numpy as np
import pandas as pd
from scipy import stats

# Wilcoxonの符号付順位検定
wilcoxon_results = []
methods = df_all['Method_Label'].unique()
num_comparisons = sum(1 for _ in combinations(methods, 2))

for axis in ['X', 'Y', 'Z']:
    for method1, method2 in combinations(methods, 2):
        data1 = df_all[df_all['Method_Label'] == method1][axis]
        data2 = df_all[df_all['Method_Label'] == method2][axis]
        stat, p_value = stats.wilcoxon(data1, data2)
        
        # 効果量rの計算
        z_value = stats.norm.ppf(p_value / 2)
        n = len(data1)
        # print(n)
        effect_size_r = z_value / np.sqrt(n)
        
        wilcoxon_results.append({
            'Axis': axis,
            'Comparison': f'{method1} vs {method2}',
            'stat': stat,
            'p_value': p_value,
            'effect_size_r': effect_size_r
        })

# 結果をデータフレームに変換
df_wilcoxon_results = pd.DataFrame(wilcoxon_results)

# ボンフェローニ補正
df_wilcoxon_results['p_value_bonferroni'] = df_wilcoxon_results['p_value'] * num_comparisons
df_wilcoxon_results['p_value_bonferroni'] = df_wilcoxon_results['p_value_bonferroni'].apply(lambda x: min(x, 1.0))

# 有意性の判断
df_wilcoxon_results['significant'] = df_wilcoxon_results['p_value_bonferroni'] < 0.05

# ボンフェローニ補正結果をCSVファイルとして保存
output_file_bonferroni = 'wilcoxon_bonferroni_results.csv'
df_wilcoxon_results.to_csv(output_file_bonferroni, index=False)

print(f"Wilcoxonの符号付順位検定結果（ボンフェローニ補正適用）が {output_file_bonferroni} に保存されました。")

# 結果の一部を表示
df_wilcoxon_results


Wilcoxonの符号付順位検定結果（ボンフェローニ補正適用）が wilcoxon_bonferroni_results.csv に保存されました。


,Axis,Comparison,stat,p_value,effect_size_r,p_value_bonferroni,significant
0,X,音無し vs 離散指数,126.0,6.310856e-09,-0.749849,6.310856e-08,True
1,X,音無し vs 連続指数,108.0,2.836109e-09,-0.766956,2.836109e-08,True
2,X,音無し vs 連続線形,699.0,1.118109e-01,-0.205282,1.000000e+00,False
3,X,音無し vs 離散線形,793.0,3.691239e-01,-0.115946,1.000000e+00,False
4,X,離散指数 vs 連続指数,839.0,5.758329e-01,-0.072229,1.000000e+00,False
5,X,離散指数 vs 連続線形,159.0,2.615939e-08,-0.718486,2.615939e-07,True
6,X,離散指数 vs 離散線形,107.0,2.711482e-09,-0.767906,2.711482e-08,True
7,X,連続指数 vs 連続線形,197.0,1.252722e-07,-0.682372,1.252722e-06,True
8,X,連続指数 vs 離散線形,145.0,1.441083e-08,-0.731792,1.441083e-07,True
9,X,連続線形 vs 離散線形,767.0,2.759259e-01,-0.140656,1.000000e+00,False


In [9]:
# 効果量が -0.5 未満または 0.5 以上の行をフィルタリング
filtered_results = df_wilcoxon_results[
    (df_wilcoxon_results['effect_size_r'] < -0.5) | (df_wilcoxon_results['effect_size_r'] > 0.5)
]

# 結果を表示
filtered_results


,Axis,Comparison,stat,p_value,effect_size_r,p_value_bonferroni,significant
0,X,音無し vs 離散指数,126.0,6.310856e-09,-0.749849,6.310856e-08,True
1,X,音無し vs 連続指数,108.0,2.836109e-09,-0.766956,2.836109e-08,True
5,X,離散指数 vs 連続線形,159.0,2.615939e-08,-0.718486,2.615939e-07,True
6,X,離散指数 vs 離散線形,107.0,2.711482e-09,-0.767906,2.711482e-08,True
7,X,連続指数 vs 連続線形,197.0,1.252722e-07,-0.682372,1.252722e-06,True
8,X,連続指数 vs 離散線形,145.0,1.441083e-08,-0.731792,1.441083e-07,True
10,Y,音無し vs 離散指数,377.0,7.477971e-05,-0.511304,7.477971e-04,True
